In [ ]:
!pip install pykep
!pip install -U TLE-tools
!pip install astropy

In [ ]:
import random
import bisect
import numpy
import scipy
import copy
from datetime import datetime
from datetime import timedelta
# -- for debris
import math
import csv
from google.colab import files
from google.colab import drive
from tletools import TLE
from astropy import constants
import pykep
# -- for ploting
import datetime
import json
import time
import networkx as nx
from matplotlib import pyplot as plt

# Genetic algorithm
The implementation uses the inver-over genetic operator to optimize the static sequence of debris based on the transference cost of the arcs.

Also, the implementation uses **index_frozen** to model the already deorbited debris.

In [ ]:
class GA:
  def __init__(self, population, fn_fitness, subpath_fn_fitness=None):
    self.population = population
    self.index_frozen = -1
    self.fitnesses = [] # fitness for each individual in population
    self.fn_fitness = fn_fitness # fitness function for the whole path
    self.subpath_fn_fitness = subpath_fn_fitness # fitness function for a subpath

  # freezes a debris in all individuals
  def freeze_first(self, frozen):
    self.index_frozen += 1
    for i in range(len(self.population)):
      del self.population[i][self.population[i].index(frozen)]
      self.population[i].insert(self.index_frozen, frozen)

  # decay a debris in all individuals
  def decay(self, decayed_debris):
    for i in range(len(self.population)):
      for x in decayed_debris:
        if x in self.population[i]:
          del self.population[i][self.population[i].index(x)]

  # force a first debris for all individuals
  def startBy(self, debris):
    for i in range(len(self.population)):
      pos = self.population[i].index(debris)
      self.population[i] = self.population[i][pos:] + self.population[i][:pos]

  # returns the best individual
  def getBest(self):
    self.fit_population()
    best = min(self.fitnesses)
    return self.population[self.fitnesses.index(best)]

  # run the inverover to optimize the static case
  """
  tinv : int : number of iterations
  feach : int : milestone to run kopt on the population
  runkopt : int : iterations of kopt
  forn : int : how many of the best individuals goes to kopt
  """
  def run_inverover(self, tinv=1000, feach=1000, runkopt=100, forn=None):
    self.fit_population()
    self.inver_over(tinv, feach, runkopt, forn)
    self.fit_population()
    best = min(self.fitnesses)
    return self.population[self.fitnesses.index(best)]

  # select a random element of the population
  def selectUniform(self):
    return self.population[random.randrange(0, len(self.population))]

  # calculate the fitness for all individuals
  def fit_population(self):
    if self.index_frozen >= 0:
      self.fitnesses = list(map(lambda x: self.subpath_fn_fitness(x[self.index_frozen:]), self.population))
    else:
      self.fitnesses = list(map(lambda x: self.fn_fitness(x), self.population))

  # run the stochastic kopt for the population
  """
  permuts : int : number of iterations
  elite : int : how many of the best shoud be processed
  """
  def koptStochastic(self, permuts=100, elite=None):
    indexes = range(len(self.population))
    if elite is not None:
      indexes = numpy.array(self.fitnesses).argsort()[:elite]

    for x in indexes:
      indv = self.population[x]
      useds = {}
      changed = False

      for _ in range(0, permuts):
        valid = False
        while not valid:
          i = random.randrange(self.index_frozen+1, len(indv))
          j = i

          while j == i: j = random.randrange(self.index_frozen+1, len(indv))

          if (i, j) not in useds:
            valid = True

        useds[(i, j)] = True

        if j < i:
          temp = i
          i = j
          j = temp

        if self.subpath_fn_fitness(list(reversed(indv[i:j+1]))) < self.subpath_fn_fitness(indv[i:j+1]):
          changed = True
          indv = indv[0:i] + list(reversed(indv[i:j+1])) + indv[j+1:]

      if changed:
        self.population[x] = indv
        self.fitnesses[x] = self.subpath_fn_fitness(indv[self.index_frozen+1:])

  # run the ranged kopt for one individual
  """
  indv : array : the individual
  maxrange : int : the range of analysis around the individual
  """
  def ranged2opt(self, indv, maxrange=10):
    ranger = indv[len(indv)-maxrange:] + indv[self.index_frozen+1: self.index_frozen+maxrange+2]
    if len(set(ranger)) != len(ranger):
      return indv

    fit = self.subpath_fn_fitness(ranger)
    changed = True

    while changed:
      changed = False
      for i in range(len(ranger)):
        for j in range(len(ranger)):
          new_ranger = ranger[0:i] + list(reversed(ranger[i:j+1])) + ranger[j+1:]
          new_fit = self.subpath_fn_fitness(new_ranger)
          if new_fit < fit:
            fit = new_fit
            ranger = new_ranger
            changed = True
            break
        if changed:
          break

    indv[len(indv)-maxrange:] = ranger[:maxrange]
    indv[self.index_frozen+1: self.index_frozen+maxrange+2] = ranger[maxrange:]

    return indv

  # run the inverover for the population
  """
  tinv : int : number of iterations
  feach : int : milestone to run kopt on the population
  runkopt : int : iterations of kopt
  forn : int : how many of the best individuals goes to kopt
  """
  def inver_over(self, tinv, feach, runkopt, forn):
    for w in range(tinv):

      if w % feach == 0:
        self.koptStochastic(runkopt, forn)

      for i in range(len(self.population)):
        tmp = self.population[i]
        c1 = tmp[random.randrange(0, len(tmp))]

        changed = False

        while True:
          sl = self.population[i]
          c2 = c1

          while sl == self.population[i]: sl = self.selectUniform()
          c2 = sl[(sl.index(c1) + 1) % len(sl)]

          pos_c1 = tmp.index(c1)
          pos_c2 = tmp.index(c2)

          # if the genes are adjacent
          if c2 in [ tmp[pos_c1-1], tmp[(pos_c1 + 1) % len(tmp)] ]:
            break
          # elif and else reverse a subset of chromosome
          elif pos_c2 > pos_c1:
            changed = True
            c1 = tmp[(pos_c2 + 1) % len(tmp)]
            tmp = tmp[:pos_c1+1] + list(reversed(tmp[pos_c1+1:pos_c2+1])) + tmp[pos_c2+1:]
          else:
            changed = True
            c1 = tmp[pos_c2-1]
            inverted = list(reversed(tmp[pos_c1:] + tmp[:pos_c2]))
            div_pos = len(tmp)-pos_c1
            tmp = inverted[div_pos:] + tmp[pos_c2:pos_c1] + inverted[:div_pos]

        if changed:
          fit_tmp = self.fn_fitness(tmp)
          if fit_tmp < self.fitnesses[i]:
            self.population[i] = tmp
            self.fitnesses[i] = fit_tmp

# Problem instance
The active debris removal problem is going to be modeled as a complex variant of Traveling Salesman Problem (TSP), the time-dependent TSP (TDTSP).

The debris are the nodes and the dynamic transference trajectories are the edges.

Also, the Max Open Walk is used to find for the optimized subpath.

In [ ]:
class StaticDebrisTSP:
  mu = 398600800000000 # gravitational parameter of earth
  re = 6378100 # radius of earth

  def __init__(self, debris=[], weight_matrix=[], reward_matrix=[], path_size=0, population_size=100, epoch=None, hohmanncost=False):
    self.index_frozen = -1
    self.debris = debris # the debris cloud
    self.reward_matrix = reward_matrix # the removal reward per debris
    self.kepler_elements = [] # kepler elements of the debris
    self.decayed_debris = [] # decayed debris
    self.hohmanncost=hohmanncost # if the cost is calculated with hohmann

    if epoch is not None:
      self.epoch = epoch
    else:
      epoch = pykep.epoch_from_string("2021-06-11 00:06:09")

    is_matrix = len(weight_matrix) != 0

    # size of a indivual
    self.size = path_size if path_size != 0 else (len(weight_matrix) if is_matrix else len(debris))

    # random population that will be used just as an input for the GA
    self.population = []
    for i in range(0, population_size):
      self.population.append(random.sample(range(0, self.size), self.size))

    # eighter receive the weight matrix or calculate it
    if is_matrix:
      self.fitness_matrix = weight_matrix

    else:
      # remove decayed debris
      i = 0
      count = 0
      qtd_decayed = 0
      while count < self.size:
        if i >= len(debris):
          break

        try:
          self.kepler_elements.append(debris[i].osculating_elements(self.epoch))
          count += 1
        except:
          self.decayed_debris.append(i)
          qtd_decayed += 1
        i += 1

      print('Decayed debris ', qtd_decayed, 'Total ', len(self.kepler_elements))
      if len(self.kepler_elements) < self.size:
        raise BaseException('Insuficient size')

      # fitness matrix
      self.fitness_matrix = numpy.zeros((self.size, self.size))
      for i in range(0, self.size):
        for j in range(0, self.size):
          if self.hohmanncost:
            self.fitness_matrix[i][j] = StaticDebrisTSP.MYhohmann_impulse_aprox(self.kepler_elements[i], self.kepler_elements[j], self.epoch)
          else:
            try:
              self.fitness_matrix[i][j] = pykep.phasing.three_impulses_approx(debris[i], debris[j], self.epoch, self.epoch)
            except:
              d1 = self.kepler_elements[i]
              d2 = self.kepler_elements[j]
              self.fitness_matrix[i][j] = StaticDebrisTSP.MYthree_impulse_aprox(d1[0],d1[1],d1[2],d1[3],d2[0],d2[1],d2[2],d2[3],StaticDebrisTSP.mu)
          

  # freezes the first element
  def freeze_first(self):
    self.index_frozen += 1

  # returns if all debris were removed
  def all_frozen(self):
    return self.index_frozen >= (self.size-1-len(self.decayed_debris))

  # transform the debris kepler elements to certain epoch
  """
  dt_epoch : datetime : the target epoch
  indexes : array : the debris that should be transformed
  """
  def to_epoch(self, dt_epoch, indexes):
    new_epoch = pykep.epoch_from_string(dt_epoch.strftime(FMT))
    ranger = [x for x in range(0, self.size) if x in indexes]
    
    self.kepler_elements = list(numpy.zeros(self.size))
    for j in ranger:
      try:
        self.kepler_elements[j] = debris[j].osculating_elements(new_epoch)
      except:
        self.decayed_debris.append(j)

    for x in self.decayed_debris:
      if x in ranger:
        del ranger[ranger.index(x)]

    for i in ranger:
      for j in ranger:
        if self.hohmanncost:
          self.fitness_matrix[i][j] = StaticDebrisTSP.MYhohmann_impulse_aprox(self.kepler_elements[i], self.kepler_elements[j], new_epoch)
        else:
          try:
            self.fitness_matrix[i][j] = pykep.phasing.three_impulses_approx(debris[i], debris[j], new_epoch, new_epoch)
          except:
            d1 = self.kepler_elements[i]
            d2 = self.kepler_elements[j]
            self.fitness_matrix[i][j] = StaticDebrisTSP.MYthree_impulse_aprox(d1[0],d1[1],d1[2],d1[3],d2[0],d2[1],d2[2],d2[3],StaticDebrisTSP.mu)

    for x in self.decayed_debris:
      if x in indexes:
        del indexes[indexes.index(x)]

    return indexes

  # fitness is the sum cost to travel between each I and I+1 plus the last to initial
  def fitness(self, solution):
    fit = 0
    for i in range(0, self.size-1):
      fit += self.fitness_matrix[solution[i]][solution[i+1]]

    fit += self.fitness_matrix[solution[self.size-1]][solution[0]]
    return fit

  # partial fitness is the sum cost to travel between each I and I+1
  def partialFitness(self, part):
    fit = 0
    for i in range(0, len(part)-1):
      fit += self.fitness_matrix[part[i]][part[i+1]]

    return fit

  # reward is the sum reward of the debris in the solution
  def reward(self, solution):
    reward = 0
    for i in range(0, len(solution)):
      reward += self.reward_matrix[solution[i]]

    return reward

  # estimate the duration of a solution
  def duration(self, solution):
    duration = 0
    for i in range(0, len(solution)-1):
      duration += self.transferDuration(solution[i], solution[i+1], StaticDebrisTSP.mu)

    return duration

  # fitness TD is the fitness function for a timedependent solution
  def fitnessTD(self, solution):
    if len(solution) < 2:
      return 0
      
    fit = 0
    for i in range(0, len(solution)-1):
      epoch = pykep.epoch_from_string((solution[i+1][0]).strftime(FMT))
      if self.hohmanncost:
        d1 = debris[solution[i][1]].osculating_elements(epoch)
        d2 = debris[solution[i+1][1]].osculating_elements(epoch)
        fit += StaticDebrisTSP.MYhohmann_impulse_aprox(d1, d2, epoch)
      else:
        try:
          fit += pykep.phasing.three_impulses_approx(debris[solution[i][1]], debris[solution[i+1][1]], epoch, epoch)
        except:
          d1 = debris[solution[i][1]].osculating_elements(epoch)
          d2 = debris[solution[i+1][1]].osculating_elements(epoch)
          fit += StaticDebrisTSP.MYthree_impulse_aprox(d1[0],d1[1],d1[2],d1[3],d2[0],d2[1],d2[2],d2[3],StaticDebrisTSP.mu)

    return fit

  # duration TD is the duration estimate for a timedependent solution
  def durationTD(self, solution):
    duration = 0
    for i in range(0, len(solution)-1):
      duration += (solution[i+1][0] - solution[i][0]).total_seconds() # seconds waiting for right epoch

      epoch = pykep.epoch_from_string(solution[i+1][0].strftime(FMT))

      duration += self.transferDurationTD(solution[i][1], solution[i+1][1], epoch, epoch, StaticDebrisTSP.mu)

    return duration

  # reward TD is the reward function for a timedependent solution
  def rewardTD(self, solution):
    reward = 0
    for i in range(0, len(solution)):
      reward += self.reward_matrix[solution[i][1]]

    return reward
  
  # estimate the duration of a transfer (Hohmann) in seconds
  def transferDuration(self, d1, d2, u):
    d1_semi_major_axis = self.kepler_elements[d1][0]
    d2_semi_major_axis = self.kepler_elements[d2][0]
    transfer_semi_major_axis = (d1_semi_major_axis + d2_semi_major_axis) / 2
    time_of_transfer = math.pi * math.sqrt((transfer_semi_major_axis**3) / u)
    return time_of_transfer

  # estimate the duration of a transfer (Hohmann) in seconds in a certain epoch
  def transferDurationTD(self, d1, d2, epoch1, epoch2, u):
    kepler1 = debris[d1].osculating_elements(epoch1)
    kepler2 = debris[d2].osculating_elements(epoch2)
    d1_semi_major_axis = kepler1[0]
    d2_semi_major_axis = kepler2[0]
    transfer_semi_major_axis = (d1_semi_major_axis + d2_semi_major_axis) / 2
    time_of_transfer = math.pi * math.sqrt((transfer_semi_major_axis**3) / u)
    return time_of_transfer

  # find the constrained embedded maximal rewardable path in a solution
  def maxOpenWalk(self, solution, cost_limit=1000, time_limit=31536000):
    # calculate transferences
    transfers = []
    durations = []
    for i in range(0, len(solution)-1):
      sol_i = solution[i]
      sol_j = solution[i+1]
      transfers.append(self.fitness_matrix[sol_i][sol_j])
      durations.append(self.transferDuration(sol_i, sol_j, StaticDebrisTSP.mu))

    # calculate the maximal open walks starting at each arc
    maxWalks = []
    for i in range(0, len(transfers)):
      cost = transfers[i]
      duration = durations[i]
      walk = [i]

      for j in range(i+1, len(transfers)):
        if (cost + transfers[j]) > cost_limit or (duration + durations[j]) > time_limit:
          break;
        else:
          cost += transfers[j]
          duration += durations[j]
          walk.append(j)

      nodes = []
      reward = 0
      for a in range(0, len(walk)):
        arc = walk[a]
        if solution[arc] not in nodes:
          nodes.append(solution[arc])
          reward += self.reward_matrix[solution[arc]]
        nodes.append(solution[arc+1])
        reward += self.reward_matrix[solution[arc+1]]

      maxWalks.append({'walk': nodes, 'cost': cost, 'duration': duration, 'reward': reward})

    # find the biggest open walk
    w = 0
    for i in range(1, len(maxWalks)):
      if maxWalks[i]['reward'] > maxWalks[w]['reward']:
        w = i

    return maxWalks[w]

  # find the constrained embedded maximal rewardable path in a timedependent solution
  def maxOpenWalkTD(self, solution, cost_limit=1000, time_limit=31536000):
    # calculate transferences
    transfers = []
    durations = []
    for i in range(0, len(solution)-1):
      epoch = pykep.epoch_from_string((solution[i+1][0]).strftime(FMT))
      sol_i = solution[i][1]
      sol_j = solution[i+1][1]

      duration = (solution[i+1][0] - solution[i][0]).total_seconds() # seconds waiting for right epoch
      duration += self.transferDurationTD(sol_i, sol_j, epoch, epoch, StaticDebrisTSP.mu)
      durations.append(duration)

      if self.hohmanncost:
        d1 = debris[sol_i].osculating_elements(epoch)
        d2 = debris[sol_j].osculating_elements(epoch)
        transfers.append(StaticDebrisTSP.MYhohmann_impulse_aprox(d1, d2, epoch))
      else:
        try:
          transfers.append(pykep.phasing.three_impulses_approx(debris[sol_i], debris[sol_j], epoch, epoch))
        except:
          d1 = debris[sol_i].osculating_elements(epoch)
          d2 = debris[sol_j].osculating_elements(epoch)
          transfers.append(StaticDebrisTSP.MYthree_impulse_aprox(d1[0],d1[1],d1[2],d1[3],d2[0],d2[1],d2[2],d2[3],StaticDebrisTSP.mu))

    # calculate the maximal open walks starting at each arc
    maxWalks = []
    for i in range(0, len(transfers)):
      cost = transfers[i]
      duration = durations[i]
      walk = [i]

      for j in range(i+1, len(transfers)):
        if (cost + transfers[j]) > cost_limit or (duration + durations[j]) > time_limit:
          break;
        else:
          cost += transfers[j]
          duration += durations[j]
          walk.append(j)

      nodes = []
      reward = 0
      for a in range(0, len(walk)):
        arc = walk[a]
        if solution[arc] not in nodes:
          nodes.append(solution[arc])
          reward += self.reward_matrix[solution[arc][1]]
        nodes.append(solution[arc+1])
        reward += self.reward_matrix[solution[arc+1][1]]

      maxWalks.append({'walk': nodes, 'cost': cost, 'duration': duration, 'reward': reward})

    # find the biggest open walk
    w = 0
    for i in range(1, len(maxWalks)):
      if maxWalks[i]['reward'] > maxWalks[w]['reward']:
        w = i

    return maxWalks[w]

  # estimate the hohmann cost for a transfer between two debris
  # kepler elements order: a,e,i,W,w,M
  def MYhohmann_impulse_aprox(kepler1, kepler2):
    if kepler1 == kepler2:
      return 0
    
    d1 = math.sqrt(StaticDebrisTSP.mu/kepler1[0]) * (math.sqrt((2*kepler2[0]) / (kepler1[0]+kepler2[0])) - 1)
    d2 = math.sqrt(StaticDebrisTSP.mu/kepler2[0]) * (- math.sqrt((2*kepler1[0]) / (kepler1[0]+kepler2[0])) + 1)
    dv = abs(d1 + d2)

    re = - StaticDebrisTSP.mu / (2 * (StaticDebrisTSP.re + kepler2[0]))
    rvi = math.sqrt(2 * ( (StaticDebrisTSP.mu / (StaticDebrisTSP.re + kepler2[0])) + re))
    romega = abs(math.degrees(kepler2[2]) - math.degrees(kepler1[2]))
    rdv = 2 * rvi * math.sin(romega/2)
    
    return abs(dv + rdv)

  # estimate the edelbaum cost for a transfer between two debris
  # this implementation replaces the pykep implementation, since pykep throws an exception for decayed debris
  def MYthree_impulse_aprox(a1, e1, i1, W1, a2, e2, i2, W2, mu):
    # radius of apocenter/pericenter starting orbit (ms)
    ra1 = a1 * (1 + e1);
    ra2 = a2 * (1 + e2);
    rp1 = a1 * (1 - e2);
    rp2 = a2 * (1 - e2);

    # relative inclination between orbits
    cosiREL = math.cos(i1) * math.cos(i2) + math.sin(i1) * math.sin(i2) * math.cos(W1 - W2)

    # Strategy is Apocenter-Pericenter
    if ra1 > ra2:
      Vi = math.sqrt(mu * (2.0 / ra1 - 1.0 / a1));
      Vf = math.sqrt(mu * (2.0 / ra1 - 2.0 / (rp2 + ra1)));

      # Change Inclination + pericenter change
      DV1 = math.sqrt(Vi * Vi + Vf * Vf - 2.0 * Vi * Vf * cosiREL);
      # Apocenter Change
      DV2 = math.sqrt(mu) * abs(math.sqrt(2.0 / rp2 - 2.0 / (rp2 + ra1)) - math.sqrt(2.0 / rp2 - 1.0 / a2));

      return (DV1 + DV2)
    
    # Strategy is Pericenter-Apocenter
    else:
      Vi = math.sqrt(mu * ((2 / ra2) - (2 / (rp1 + ra2))));
      Vf = math.sqrt(mu * ((2 / ra2) - (1 / a2)));

      # Apocenter Raise
      DV1 = math.sqrt(mu) * abs(math.sqrt((2 / rp1) - (2 / (rp1 + ra1))) - math.sqrt((2 / rp1) - (2 / (rp1 + ra2))));
      # Change Inclination + apocenter change
      DV2 = math.sqrt(abs((Vi * Vi) + (Vf * Vf) - (2 * Vi * Vf * cosiREL)));

      return (DV1 + DV2)

# Instance loading
The instances can be downloaded at SATCAT site.

It is necessary to use a TXT file (TLE file) to get the debris names, codes and kepler elements, and a CSV file for the debris RCS (reward).

In [ ]:
deb_file = 'fengyun-1c-debris'
debris = pykep.util.read_tle(tle_file=deb_file+'.txt', with_name=True)
with open(deb_file+'.txt') as f:
    tle_string = ''.join(f.readlines())

tle_lines = tle_string.strip().splitlines()
tle_elements = [tle_lines[i:i + 3] for i in range(0, len(tle_lines), 3)] #split in array of debris
debris_tle = [TLE.from_lines(*tle_elements[i]) for i in range(0, len(tle_elements))]

with open(deb_file+'.csv', newline='') as csvfile:
    satcat = list(csv.reader(csvfile))

In [ ]:
# extract the reward for each debris
areaDebris = []
norad_index = satcat[0].index('NORAD_CAT_ID')
rcs_index = satcat[0].index('RCS')
for i in range(0, len(debris)):
  rcs = 0
  for j in range(1, len(satcat)):
    if (debris_tle[i].norad == satcat[j][norad_index]):
      if (satcat[j][rcs_index]):
        rcs = float(satcat[j][rcs_index])
      break
  areaDebris.append(rcs)

# Solution
Here the actual solution is generated.

An interpolated tree search is performed to enhance the static to a time dependent solution.

In [ ]:
start_epoch = "2021-06-11 00:06:09"
FMT = '%Y-%m-%d %H:%M:%S'
steps = int((24 * 60) / 10) * 7 # in days
step_size = timedelta(minutes=10)
removal_time = timedelta(days=1) # time taken to deorbit a debris
winsize = 10 # range for the kopt

for _ in range(10):
  t0 = datetime.datetime.now() # to track time elapsed
  epoch = datetime.datetime.strptime(start_epoch, FMT)

  # generate the ga and problem instance
  problem = StaticDebrisTSP(epoch=pykep.epoch_from_string(start_epoch), hohmanncost=False, debris=debris, reward_matrix=areaDebris, path_size=size, population_size=100)
  ga = GA(population=problem.population, fn_fitness=problem.fitness, subpath_fn_fitness=problem.partialFitness)

  # generate the static solution
  curr_solution = ga.run_inverover(tinv=20000, feach=1000, runkopt=100, forn=5)
  curr_fit = problem.partialFitness(curr_solution)
  print('initial fit: '+str(curr_fit))

  # find the static max open walk
  path = problem.maxOpenWalk(curr_solution, 1000, 60*60*24*365) # 1km/s and 1 year
  
  # make the population start by best starting debris, and get the best then
  ga.startBy(path['walk'][0])
  curr_solution = ga.getBest()
  curr_fit = problem.partialFitness(curr_solution)
  print('secondal fit: '+str(curr_fit))

  # use the first debris for the time dependent solution
  solution = [(epoch, curr_solution[0])]
  problem.freeze_first()
  ga.freeze_first(curr_solution[0])

  while not problem.all_frozen():
    i = problem.index_frozen
    
    # run ranged kopt to optimize the current part of solution
    if i > 0 and (i < len(curr_solution)-1):
      curr_solution[i:i+winsize+1] = problem.to_epoch(epoch, curr_solution[i:i+winsize+1])
      curr_solution[-(winsize+1):] = problem.to_epoch(epoch, curr_solution[-(winsize+1):])
      ga.decay(problem.decayed_debris)
      curr_solution = ga.ranged2opt(curr_solution, winsize)
    
    # get the next transference to be performed
    transition = curr_solution[i:i+2]
    
    # validates if the debris in this transference are going to decay in during the interpolation
    transition = problem.to_epoch(epoch + (step_size * steps), transition)
    if len(transition) < 2:
      curr_solution[i:i+2] = transition
      ga.decay(problem.decayed_debris)
      continue

    # calculate the costs of the transference for the interpolation range
    epoch_aux = epoch
    x = []
    y = []
    for j in range(0, steps):
      problem.to_epoch(epoch, transition)
      x.append(j)
      y.append(problem.partialFitness(transition))
      epoch += step_size

    # get the minimal cost point in the interpolated function
    interpolator = scipy.interpolate.interp1d(x, y, kind='cubic')
    xnew = numpy.linspace(0, steps-1, num=steps*3, endpoint=True) # num = precision
    least = numpy.argmin(interpolator(xnew))

    # get the epoch of the minimal cost transference
    epoch = epoch_aux + (step_size * xnew[least])

    # append to the time dependent solution
    solution.append((epoch, curr_solution[i+1]))

    # pushes the current epoch to after the deorbitation process
    pykep_epoch = pykep.epoch_from_string(epoch.strftime(FMT))
    transfer_duration = timedelta(seconds=problem.transferDurationTD(curr_solution[i], curr_solution[i+1], pykep_epoch, pykep_epoch, StaticDebrisTSP.mu))
    epoch += removal_time + transfer_duration

    # freezes the deorbited debris
    problem.freeze_first()
    ga.freeze_first(curr_solution[i+1])

  t1 = datetime.datetime.now()

In [ ]:
# instance results
print(solution)
print('elapsed time: '+ str(t1 - t0))
print('fit: ' + str(problem.fitnessTD(solution)))
print('dur: ' + str(problem.durationTD(solution)/60/60/24) + ' days')
print('rew: ' + str(problem.rewardTD(solution)))

# constrained (best mission) results
path = problem.maxOpenWalkTD(solution, 1000, 60*60*24*365) # 1km/s and 1 year
print(path)
print('walk ' + str(len(path['walk'])))
print('w_cost ' + str(path['cost']))
print('w_rew ' + str(path['reward']))
print('w_dur ' + str(path['duration']/60/60/24) + ' days')

# Bibliography

**Instances**

TLE Derbis: https://celestrak.com/NORAD/elements/

RCS: https://celestrak.com/satcat/search.php - LEGACY text

Format: https://celestrak.com/satcat/satcat-format.php

**Used Libs**

https://esa.github.io/pykep/documentation/phasing.html#pykep.phasing.three_impulses_approx

**Reference source codes**

https://github.com/esa/pagmo/blob/master/src/problem/base_tsp.cpp
https://github.com/esa/pagmo/blob/master/src/algorithm/inverover.cpp

https://stackoverflow.com/questions/47982604/hamiltonian-path-using-python/47985349
https://github.com/esa/pagmo/blob/80281d549c8f1b470e1489a5d37c8f06b2e429c0/src/util/neighbourhood.cpp

https://github.com/esa/pagmo/blob/80281d549c8f1b470e1489a5d37c8f06b2e429c0/PyGMO/problem/_tsp.py
https://github.com/esa/pagmo/blob/80281d549c8f1b470e1489a5d37c8f06b2e429c0/src/problem/base_tsp.cpp
https://github.com/esa/pagmo/blob/80281d549c8f1b470e1489a5d37c8f06b2e429c0/src/problem/tsp.cpp
https://github.com/esa/pagmo/blob/80281d549c8f1b470e1489a5d37c8f06b2e429c0/src/problem/tsp_cs.cpp
https://github.com/esa/pagmo/blob/80281d549c8f1b470e1489a5d37c8f06b2e429c0/src/problem/tsp_ds.cpp
https://github.com/esa/pykep/blob/2e1c97bea138d2c125d6695e7662991e6da30203/include/keplerian_toolbox/core_functions/three_impulses_approximation.hpp

**Reference physics**

https://en.wikipedia.org/wiki/Hohmann_transfer_orbit

https://en.wikipedia.org/wiki/Kepler%27s_laws_of_planetary_motion#Third_law

https://en.wikipedia.org/wiki/Orbital_period

https://space.stackexchange.com/questions/35166/how-to-find-t₀-and-other-parameters-from-a-tle-to-calculate-an-approximate-mean/35190#35190

https://space.stackexchange.com/questions/18289/how-to-get-semi-major-axis-from-tle

https://ai-solutions.com/_freeflyeruniversityguide/hohmann_transfer.htm